In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler

# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
import jedi, parso
from coeditor.dataset import TokenizedEditDataset
from pprint import pprint


In [3]:
import jedi.api.classes

s = jedi.Script(
"""\
from dataclasses import dataclass
@dataclass
class Foo:
    bar: int
"""
)

In [4]:
defs = s.goto(3, 8)  # go to Foo directly
print(f"{len(defs)=}")
n = defs[0]
assert isinstance(n, jedi.api.classes.Name)
print(n._get_docstring_signature())

len(defs)=1
Foo(bar: int)


In [6]:
defs = s.goto(4, 6)  # first go to bar
print(f"{len(defs)=}")
n = defs[0]
assert isinstance(n, jedi.api.classes.Name)
print(n._get_docstring_signature())

len(defs)=1



In [17]:
pnode = n.parent()
assert isinstance(pnode, jedi.api.classes.Name)
pnode

<Name full_name='coeditor.c3problem.C3ProblemTokenizer', description='class C3ProblemTokenizer'>

In [9]:
for sig in n.parent()._get_signatures():
    print(sig.to_string())

C3ProblemTokenizer()


In [10]:
from coeditor.git import get_commit_history, CommitInfo

repo_root = proj_root()
# repo_root = get_dataset_dir("tiny") / "repos/test/gym"

commits = get_commit_history(repo_root, 20)
# commits = [CommitInfo("9dea81b48a2e1d8f7e7a81211c0f09f627ee61a9", (), "omit"),
#            CommitInfo("6a4cefd875f6547fc0bb43e02b14bb21d70fda9a", (), "omit")]
for c in commits[:3]:
    print(c)


CommitInfo(hash='8b7c5296eb4ca6b0b898bd216b0369226d60f406', parents=('bd89c861a6d0df7743a0e8b8351a2bc7cd9d44a4',), msg='handle trailing spaces in test.')
CommitInfo(hash='bd89c861a6d0df7743a0e8b8351a2bc7cd9d44a4', parents=('16aba227a72a38fa84d6def0ae82aeb8a152563e',), msg='More edits tests.')
CommitInfo(hash='16aba227a72a38fa84d6def0ae82aeb8a152563e', parents=('c1a8d33c4eeaf20faca18a12b759677f80dbda72',), msg='Keep items order in random_subset.')


In [11]:
from coeditor.scoped_changes import edits_from_commit_history, _tlogger
from coeditor.c3problem import C3ProblemGenerator, C3ProblemTokenizer, JediUsageAnalyzer

generator = C3ProblemGenerator()

workdir = proj_root() / "../temp-1"
# subprocess.run(["rm", "-r", workdir])
_tlogger.clear()
problems = edits_from_commit_history(
    repo_root, commits, workdir, change_processor=generator
)
print(f"{len(problems) = }")
for err, count in generator.analyzer.error_counts.items():
    print(f"({count=}): {err}")
display(_tlogger.as_dataframe())


processing commits: 100%|██████████| 19/19 [00:35<00:00,  1.87s/it]

len(problems) = 112


,name,count,avg_time,total_time
4,pre_edit_analysis,19,1.148602,21.823437
3,post_edit_analysis,19,0.129084,2.452600
1,parse_module,92,0.018053,1.660913
5,process_change,19,0.066640,1.266154
2,JModuleChange.from_modules,59,0.010731,0.633121
0,checkout,58,0.007797,0.452197


In [6]:
from coeditor.encoding import encode_diffs, _BaseTokenizer, Add_id, decode_tokens
from coeditor.c3problem import C3ProblemSimpleSplit, C3ProblemChangeDropout

prob_trans = C3ProblemChangeDropout()
encoder = C3ProblemTokenizer()
# %lprun -T output/lprof.txt -f C3ProblemTokenizer.tokenize_problem edits = [e for p in problems for e in encoder.tokenize_problem(p)]
new_probs = [p1 for p in problems for p1 in prob_trans.transform(p) if "dropout" in p1.transformations]

edits = [encoder.tokenize_problem(p) for p in new_probs]

# dset = TokenizedEditDataset.from_edits(edits)
# pretty_print_dict(dset.overall_stats())
# e = edits[4]
# print(e.show())


In [29]:
print(edits[24].show())

--------------------------------------------------------------------------------
path: coeditor.c3problem/C3ProblemGenerator.process_change
n_references: 14
total_reference_tks: 6000
project: temp-1
commit: CommitInfo(hash='10c5e6f0d8a5440b63a9a41bac37d5eca50b6d6b', parents=('59740e7a9d5b2998ca11a37f650e7b056ba4da4f',), msg='Update c3ProblemChangeDropout to v1.1.')
========Ground Truth========
 <0>:<add>             path = cs.earlier.path
 <4>:<del>                     cs.earlier().tree.type,
 <5>:<add>                     cs.earlier.tree.type,
 <6>:<del>                     cs.earlier().path,
 <7>:<add>                     cs.earlier.path,

========Main Code========
      # module: coeditor.c3problem
      class C3ProblemGenerator(ProjectChangeProcessor[C3Problem]):
          def process_change(
              self,
              pchange: JProjectChange,
              mod2usages: Mapping[ModuleName, LineUsageAnalysis],
              module_order: Sequence[ModuleName],
          ) -> Se

In [6]:
%%timeit -n 1 -r 2
analysis = UsageAnalysis(pyproj, add_implicit_rel_imports=True, record_type_usages=True)

9.82 s ± 26 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [34]:
%%timeit -n 1 -r 2
PythonProject.from_root(proj_root())

4.79 s ± 21.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [17]:
jproj = jedi.Project(proj_root())
jmodules = dict()
for f in proj_root().glob("src/**/*.py"):
    s = jedi.Script(path=f, project=jproj)
    jmodules[f] = s._module_node


In [3]:
jproj = jedi.Project(proj_root())
s = jedi.Script(path=proj_root() / "src/coeditor/usage_analysis.py")
s.goto(28, 9)


[<Name name='self', description='param self'>]